In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
# Add warehouse data
entry2_data = pd.read_csv("../data/entry_loc2.csv")
entry2_data = entry2_data.dropna()
entry2_data['HD entry'] = 1*(entry2_data['HD'] > 0)
entry2_data['LOW entry'] = 1*(entry2_data['LOW'] > 0)
entry2_data['log income'] = np.log(1 + entry2_data['income_per_capita'])
entry2_data['log population'] = np.log(1 + entry2_data['population'])
entry2_data['log hd warehouse'] = np.log(1 + entry2_data['hd warehouse distance'])
entry2_data['log low warehouse'] = np.log(1 + entry2_data['low warehouse distance'])
entry2_data = entry2_data[(entry2_data['population']>=10000) & (entry2_data['income_per_capita']>=15000)]
entry2_data = entry2_data[(entry2_data['lon'] <= -30) & (entry2_data['lat'] >= 25) ]
entry2_data = sm.add_constant(entry2_data)

/home/erichschulman/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [3]:
def contraction(params, x, p):
    # beta and x are kind of parameters. x is the empirical distribution of x?
    rho = np.clip(params[-1],-1,1)
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:, 0:k], params[0:k]) + params[-3]*p[:,1]
    util2 = np.dot(x[:, k:2*k], params[k:2*k]) + params[-2]*p[:,0]
    p = np.array([np.exp(util1)/(1+np.exp(util1)),
                    np.exp(util2)/(1+np.exp(util2))])
    
    #summarize prob
    prob01 = (1 - p[0, :])*(p[1, :]) 
    prob10 = (p[0, :])*(1-p[1, :]) 
    prob00 = (1-p[0, :])*(1-p[1, :]) 
    prob11 = (p[0, :])*(p[1, :]) 
    
    #introduce the copula
    prob01_rho = prob01*(1+rho*prob10)
    prob10_rho = prob10*(1+rho*prob01)
    prob00_rho = prob00*(1+rho*prob11)
    prob11_rho = prob11*(1+rho*prob00)
    
    return np.array( [[prob00,prob01],[prob10,prob11]] )


def contraction_map(betas, x, p):
    """final result is beliefs of firm1/firm2"""
    full_p = None
    for i in range(50):
        full_p = contraction(betas, x, p)
        
        #use bayes rule to figure out full pr?
        p1 = full_p[1,1] + full_p[1,0]
        p2 = full_p[1,1] + full_p[0,1]
        p = np.array([p1,p2]).transpose()
        
    return p

In [4]:
class BayesNashLogit(GenericLikelihoodModel):

    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = self.endog
        p = contraction_map(params, self.exog, p)
        p = np.log(contraction(params, self.exog, p))
        
        p00 = (1 - self.endog[:, 0])*(1 - self.endog[:, 1])
        p11 = self.endog[:, 0]*self.endog[:, 1]
        p10 = self.endog[:, 0] * (1 - self.endog[:, 1])
        p01 = (1 - self.endog[:, 0]) * self.endog[:, 1]

        ll = p01 * p[0,1] + p10 * p[1,0] + p00 * p[0,0] + p11 * p[1,1]
        
        return -1*ll

    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        k = int(self.exog.shape[1]/2)
        x1 = np.concatenate( (self.exog[:, 0:k], self.endog[:,1].reshape(self.endog.shape[0],1) ) ,axis=1)
        x2 = np.concatenate( (self.exog[:, k:2*k], self.endog[:,0].reshape(self.endog.shape[0],1) ),axis=1)
        params1 =  sm.Logit(self.endog[:, 0], x1).fit().params
        params2 = sm.Logit(self.endog[:, 1], x2).fit().params
        start_params = np.concatenate((params1[0:-1],params2[0:-1],[params1[-1],params2[-1],.5] ))
        print(start_params)
        return super(BayesNashLogit, self).fit(start_params=start_params, **kwds)
    
    
x = entry2_data[['const','log income','log population','log hd warehouse',
                 'const','log income','log population','log low warehouse']].copy()
y = entry2_data[['HD entry','LOW entry']]

BayesNashLogit_model = BayesNashLogit(y,x).fit()
print(BayesNashLogit_model.summary())

Optimization terminated successfully.
         Current function value: 0.516223
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.521237
         Iterations 6
[-1.09364059e+01 -5.08298059e-03  9.45608270e-01  8.33899479e-03
  4.47877722e+00 -1.15220618e+00  5.79471773e-01  5.06419108e-03
  1.33141327e+00  1.35747936e+00  5.00000000e-01]
                                BayesNashLogit Results                               
Dep. Variable:     ['HD entry', 'LOW entry']   Log-Likelihood:                -3264.1
Model:                        BayesNashLogit   AIC:                             6538.
Method:                   Maximum Likelihood   BIC:                             6568.
Date:                       Tue, 10 May 2022                                         
Time:                               08:32:48                                         
No. Observations:                       2952                                         
Df Residuals: 

/home/erichschulman/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/erichschulman/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
